In [1]:
import pandas as pd
from gensim import corpora
from gensim import models

LDA模型就可以直接对电影的全部评论文本一次性做主题分析，但是如果把文本的正面评价（好评）和负面评价（差评）混淆在一起，并且由于分词粒度的影响（否定词或程度词等，可能在一个主题下生成一些令人迷惑的词语。因此抛弃中性的评论，对好评和差评分别进行LDA主题分析是一个比较好的主题。

In [2]:
#读取经过预处理的好评数据
pos = pd.read_csv('comments_sets/pre_pos.txt', encoding = 'utf-8', header = None)
#读取经过预处理的差评数据
neg = pd.read_csv('comments_sets/pre_neg.txt', encoding = 'utf-8', header = None)

In [3]:
pos.head()

,0
0,"['喜欢', '这部', '电影', '故事', '终章', '读过', '故事']"
1,"['90', '羡慕', '一代人', '40', '年前', '观众', '影院', '里..."
2,"['托尼', '说好', '回归', '家庭', '陪伴', '家人', '最终', '选择..."
3,"['美队', '错过', '卡特', '约会', '错过', '陪伴', '卡特', '70..."
4,"['黑寡妇', '高尚', '豪迈', '英勇', '无畏', '钢铁', '侠', '并列..."


In [4]:
neg.head()

,0
0,"['一群', '好几百', '岁', '叽叽', '歪歪', '一群', '青春期', '小..."
1,"['300', '杜比', '首映', '场', '真的', '不值', '花里胡哨', '..."
2,"['前', '20', '分钟', '真的', '罗素', '兄弟', '捧', '好菜',..."
3,"['承认', '流量', '时代', '主宰世界']"
4,"['小时', '超长', 'CP', '混剪', '水准', '贼', '烂', '那种',..."


In [5]:
#定义一个分割函数，然后用 apply 广播
pos[1] = pos[0].apply(lambda s: s.split(' '))
neg[1] = neg[0].apply(lambda s: s.split(' '))

In [19]:
## 正面主题分析
pos_dict = corpora.Dictionary(pos[1])
pos_corpus = [pos_dict.doc2bow(i) for i in pos[1]]
pos_lda = models.LdaModel(pos_corpus, num_topics = 2, id2word = pos_dict)
print("\n正面评价")
for i in range(2):
	print("潜在主题%d : " %(i+1))
	print(pos_lda.print_topic(i) ) #输出每个主题


正面评价
潜在主题1 : 
0.011*"'电影'," + 0.007*"'英雄'," + 0.007*"'漫威'," + 0.006*"'复联'," + 0.005*"'里'," + 0.005*"'复仇者'," + 0.004*"'爱'," + 0.004*"'故事'," + 0.004*"['I'," + 0.004*"'太',"
潜在主题2 : 
0.016*"'漫威'," + 0.013*"'电影'," + 0.012*"'十年'," + 0.008*"'钢铁'," + 0.007*"'英雄'," + 0.007*"'宇宙'," + 0.007*"'侠'," + 0.007*"'复联'," + 0.006*"'超级'," + 0.006*"'一部',"


根据对有用特征词提取，

主题1中的高频特征词：英雄，爱，故事，太等

主题2中的高频特征词：十年，钢铁，英雄，宇宙，侠，复仇者等

综上对正面主题及其中高频特征词可以看出，复仇者联盟4的优点可以归结为：

1)钢铁侠，黑寡妇，美国队长，雷神等超级英雄，每一位都有自己的故事和背景，个人的刻画还有感情线的剧情泪点和燃点齐飞，让人赞叹。

2)对于“超级英雄片”的主题不忘初心，一晃十年，在这个急功近利的时代，他们的“匠心”显得弥足珍贵。

In [29]:
## 负面主题分析
neg_dict = corpora.Dictionary(neg[1])
neg_corpus = [neg_dict.doc2bow(i) for i in neg[1]]
neg_lda = models.LdaModel(neg_corpus, num_topics = 2, id2word = neg_dict)
print("\n负面评价")

for i in range(2):
	print("潜在主题%d : " %(i+1))
	print(neg_lda.print_topic(i) ) #输出每个主题


负面评价
潜在主题1 : 
0.012*"'电影'," + 0.009*"'真的'," + 0.009*"'漫威'," + 0.007*"'剧情'," + 0.007*"'情怀'," + 0.007*"'英雄'," + 0.006*"'小时'," + 0.006*"'队长'," + 0.006*"'太'," + 0.006*"'系列',"
潜在主题2 : 
0.017*"'电影'," + 0.009*"'小时'," + 0.007*"'故事'," + 0.006*"'一部'," + 0.006*"'漫威'," + 0.006*"'穿越'," + 0.006*"'人物'," + 0.005*"'角色'," + 0.005*"'煽情'," + 0.005*"'情怀',"


主题1的高频特征词：小时，剧情，情怀，人物等

主题2的高频特征词：剧情，故事，穿越，逻辑等

相对而言，复仇者联盟4的不足之处（差评点）主要体现于：

1)其中多少也夹杂着一点老牌卖情怀回忆，三个小时的剧情安排略为拖拉,部分关键人物只是在结尾大战关头才出来露个面，没多少镜头可以发挥。

2)剧情逻辑不严谨(如部分情节被认定违背了时间悖论）。